# **Import & connect to drive**

import all libraries and put all installations here :

In [ ]:
!pip install emoji
!pip install datasets
import nltk
nltk.download('punkt')
nltk.download('stopwords')

import os
import pandas as pd
from datasets import load_dataset
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential

import re
import emoji
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.9/457.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.1 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Connect to drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **Load data and process**

In [ ]:
# Define the paths to the CSV files
save_dir = "/content/drive/MyDrive/AD"
train_csv_path = os.path.join(save_dir, 'train.csv')

# Read CSV files into DataFrames
train_df = pd.read_csv(train_csv_path)


# Display the first few rows of each DataFrame for inspection
print("Train DataFrame:")
print(train_df.head())

# **Clean emojis**

In [ ]:
train_df

In [ ]:
train_df.count()

In [ ]:
missing_values = train_df.isna().sum()
missing_values


Functions to remove emojis

In [ ]:
def replace_emoji(sent):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', sent)

In [ ]:
def is_emoji(s):
    return s in UNICODE_EMOJI['en']
# add space near your emoji
def add_space(text):
    return ''.join(' ' + char if is_emoji(char) else char for char in text).strip()

We clean the emojis from tweet column and call it new tweet and test if there are no emojis left

Test replace_emoji function

In [ ]:
replace_emoji('😂😂😂😂😂')

In [ ]:
train_df["no_emoji"]=train_df["tweet"].apply(lambda row: replace_emoji(row))

In [ ]:
train_df

# **Clean stop words and special characters**


To lowercase

In [ ]:
train_df['no_emoji_lowercase'] = train_df['no_emoji'].apply(lambda x: x.lower())  # Convert to lowercase

In [ ]:
train_df

,tweet,label,no_emoji,no_emoji_lowercase
0,hhhhhhhhhhhhhhhhhhhhhh ana ga3ma sma3tt ach ka...,1,hhhhhhhhhhhhhhhhhhhhhh ana ga3ma sma3tt ach ka...,hhhhhhhhhhhhhhhhhhhhhh ana ga3ma sma3tt ach ka...
1,hhhh hardart jomhorha la fi3lan kadahko lach m...,1,hhhh hardart jomhorha la fi3lan kadahko lach m...,hhhh hardart jomhorha la fi3lan kadahko lach m...
2,: أتفهم العثماني وأتفهم الجدل الشعبي. الحل: حا...,1,: أتفهم العثماني وأتفهم الجدل الشعبي. الحل: حا...,: أتفهم العثماني وأتفهم الجدل الشعبي. الحل: حا...
3,كان افظل حاكم عربي',0,كان افظل حاكم عربي',كان افظل حاكم عربي'
4,ماريانو راخوي : خاص المغرب واسبانيا يشرحو للرأ...,1,ماريانو راخوي : خاص المغرب واسبانيا يشرحو للرأ...,ماريانو راخوي : خاص المغرب واسبانيا يشرحو للرأ...
...,...,...,...,...
5578,wah akhouya titiz o dikchi 3la 9ad lhal,1,wah akhouya titiz o dikchi 3la 9ad lhal,wah akhouya titiz o dikchi 3la 9ad lhal
5579,soukaina dik nhar drt follow haydato o 9lt f k...,1,soukaina dik nhar drt follow haydato o 9lt f k...,soukaina dik nhar drt follow haydato o 9lt f k...
5580,kiban liya stormy kid7ak mn hna ana,1,kiban liya stormy kid7ak mn hna ana,kiban liya stormy kid7ak mn hna ana
5581,woww ghadi tefra7 bzzf thank uu aaliaaa,0,woww ghadi tefra7 bzzf thank uu aaliaaa,woww ghadi tefra7 bzzf thank uu aaliaaa


In [ ]:
# Load the stop words from the file into a set
with open('/content/drive/MyDrive/Colab Notebooks/stop_words.txt', 'r', encoding='utf-8') as file:
    stop_words = set(file.read().split())


# Remove stop words
train_df['no_emoji_lowercase_stop_words'] = train_df['no_emoji_lowercase'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

# Display
train_df

,tweet,label,no_emoji,no_emoji_lowercase,no_emoji_lowercase_stop_words
0,hhhhhhhhhhhhhhhhhhhhhh ana ga3ma sma3tt ach ka...,1,hhhhhhhhhhhhhhhhhhhhhh ana ga3ma sma3tt ach ka...,hhhhhhhhhhhhhhhhhhhhhh ana ga3ma sma3tt ach ka...,hhhhhhhhhhhhhhhhhhhhhh ana ga3ma sma3tt ach ka...
1,hhhh hardart jomhorha la fi3lan kadahko lach m...,1,hhhh hardart jomhorha la fi3lan kadahko lach m...,hhhh hardart jomhorha la fi3lan kadahko lach m...,hhhh hardart jomhorha la fi3lan kadahko lach m...
2,: أتفهم العثماني وأتفهم الجدل الشعبي. الحل: حا...,1,: أتفهم العثماني وأتفهم الجدل الشعبي. الحل: حا...,: أتفهم العثماني وأتفهم الجدل الشعبي. الحل: حا...,: أتفهم العثماني وأتفهم الجدل الشعبي. الحل: حا...
3,كان افظل حاكم عربي',0,كان افظل حاكم عربي',كان افظل حاكم عربي',افظل حاكم عربي'
4,ماريانو راخوي : خاص المغرب واسبانيا يشرحو للرأ...,1,ماريانو راخوي : خاص المغرب واسبانيا يشرحو للرأ...,ماريانو راخوي : خاص المغرب واسبانيا يشرحو للرأ...,ماريانو راخوي : خاص المغرب واسبانيا يشرحو للرأ...
...,...,...,...,...,...
5578,wah akhouya titiz o dikchi 3la 9ad lhal,1,wah akhouya titiz o dikchi 3la 9ad lhal,wah akhouya titiz o dikchi 3la 9ad lhal,wah akhouya titiz o dikchi 3la 9ad lhal
5579,soukaina dik nhar drt follow haydato o 9lt f k...,1,soukaina dik nhar drt follow haydato o 9lt f k...,soukaina dik nhar drt follow haydato o 9lt f k...,soukaina dik nhar drt follow haydato o 9lt f k...
5580,kiban liya stormy kid7ak mn hna ana,1,kiban liya stormy kid7ak mn hna ana,kiban liya stormy kid7ak mn hna ana,kiban liya stormy kid7ak mn hna ana
5581,woww ghadi tefra7 bzzf thank uu aaliaaa,0,woww ghadi tefra7 bzzf thank uu aaliaaa,woww ghadi tefra7 bzzf thank uu aaliaaa,woww ghadi tefra7 bzzf thank uu aaliaaa


remove_special_characters and test function before applying it

In [ ]:
def remove_special_characters(text):
    # Define a pattern to match special characters (excluding alphanumeric, Arabic characters, and whitespace)
    pattern = re.compile('[^A-Za-z0-9\s\u0600-\u06FF]+', flags=re.UNICODE)

    # Use the pattern to replace special characters with an empty string
    clean_text = pattern.sub('', text)

    return clean_text

# Example usage
original_text = "واللي معاه شاء شاء وآبى^$*:;!!!!'(-è_çà)=à&&&<<<< آبى ..'   ANA MASO9ICH F HADCHI "
cleaned_text = remove_special_characters(original_text)

print("Original Text:", original_text)
print("Cleaned Text:", cleaned_text)


Original Text: واللي معاه شاء شاء وآبى^$*:;!!!!'(-è_çà)=à&&&<<<< آبى ..'   ANA MASO9ICH F HADCHI 
Cleaned Text: واللي معاه شاء شاء وآبى آبى    ANA MASO9ICH F HADCHI 


In [ ]:
train_df['no_emoji_lowercase_stop_words_characters'] = train_df['no_emoji_lowercase_stop_words'].apply(remove_special_characters)

In [ ]:
train_df_c=train_df.copy()

# **Final**

Visualize to compare

In [ ]:
train_df

,tweet,label,no_emoji,no_emoji_lowercase,no_emoji_lowercase_stop_words,no_emoji_lowercase_stop_words_characters
0,hhhhhhhhhhhhhhhhhhhhhh ana ga3ma sma3tt ach ka...,1,hhhhhhhhhhhhhhhhhhhhhh ana ga3ma sma3tt ach ka...,hhhhhhhhhhhhhhhhhhhhhh ana ga3ma sma3tt ach ka...,hhhhhhhhhhhhhhhhhhhhhh ana ga3ma sma3tt ach ka...,hhhhhhhhhhhhhhhhhhhhhh ana ga3ma sma3tt ach ka...
1,hhhh hardart jomhorha la fi3lan kadahko lach m...,1,hhhh hardart jomhorha la fi3lan kadahko lach m...,hhhh hardart jomhorha la fi3lan kadahko lach m...,hhhh hardart jomhorha la fi3lan kadahko lach m...,hhhh hardart jomhorha la fi3lan kadahko lach m...
2,: أتفهم العثماني وأتفهم الجدل الشعبي. الحل: حا...,1,: أتفهم العثماني وأتفهم الجدل الشعبي. الحل: حا...,: أتفهم العثماني وأتفهم الجدل الشعبي. الحل: حا...,: أتفهم العثماني وأتفهم الجدل الشعبي. الحل: حا...,أتفهم العثماني وأتفهم الجدل الشعبي الحل حان و...
3,كان افظل حاكم عربي',0,كان افظل حاكم عربي',كان افظل حاكم عربي',افظل حاكم عربي',افظل حاكم عربي
4,ماريانو راخوي : خاص المغرب واسبانيا يشرحو للرأ...,1,ماريانو راخوي : خاص المغرب واسبانيا يشرحو للرأ...,ماريانو راخوي : خاص المغرب واسبانيا يشرحو للرأ...,ماريانو راخوي : خاص المغرب واسبانيا يشرحو للرأ...,ماريانو راخوي خاص المغرب واسبانيا يشرحو للرأي...
...,...,...,...,...,...,...
5578,wah akhouya titiz o dikchi 3la 9ad lhal,1,wah akhouya titiz o dikchi 3la 9ad lhal,wah akhouya titiz o dikchi 3la 9ad lhal,wah akhouya titiz o dikchi 3la 9ad lhal,wah akhouya titiz o dikchi 3la 9ad lhal
5579,soukaina dik nhar drt follow haydato o 9lt f k...,1,soukaina dik nhar drt follow haydato o 9lt f k...,soukaina dik nhar drt follow haydato o 9lt f k...,soukaina dik nhar drt follow haydato o 9lt f k...,soukaina dik nhar drt follow haydato o 9lt f k...
5580,kiban liya stormy kid7ak mn hna ana,1,kiban liya stormy kid7ak mn hna ana,kiban liya stormy kid7ak mn hna ana,kiban liya stormy kid7ak mn hna ana,kiban liya stormy kid7ak mn hna ana
5581,woww ghadi tefra7 bzzf thank uu aaliaaa,0,woww ghadi tefra7 bzzf thank uu aaliaaa,woww ghadi tefra7 bzzf thank uu aaliaaa,woww ghadi tefra7 bzzf thank uu aaliaaa,woww ghadi tefra7 bzzf thank uu aaliaaa


In [ ]:
train_df = train_df.drop(['tweet', 'no_emoji', 'no_emoji_lowercase','no_emoji_lowercase_stop_words'],axis=1)

train_df.head()


,label,no_emoji_lowercase_stop_words_characters
0,1,hhhhhhhhhhhhhhhhhhhhhh ana ga3ma sma3tt ach ka...
1,1,hhhh hardart jomhorha la fi3lan kadahko lach m...
2,1,أتفهم العثماني وأتفهم الجدل الشعبي الحل حان و...
3,0,افظل حاكم عربي
4,1,ماريانو راخوي خاص المغرب واسبانيا يشرحو للرأي...


In [ ]:
train_df = train_df.rename(columns={'no_emoji_lowercase_stop_words_characters' : 'clean_tweet'})

In [ ]:
train_df.head()


,label,clean_tweet
0,1,hhhhhhhhhhhhhhhhhhhhhh ana ga3ma sma3tt ach ka...
1,1,hhhh hardart jomhorha la fi3lan kadahko lach m...
2,1,أتفهم العثماني وأتفهم الجدل الشعبي الحل حان و...
3,0,افظل حاكم عربي
4,1,ماريانو راخوي خاص المغرب واسبانيا يشرحو للرأي...


# **Save in a csv**

In [ ]:
train_df.to_csv('/content/drive/MyDrive/AD/clean_data.csv', index=False)

# **Model training for bert - 5 it**

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
import torch
from torch.utils.data import TensorDataset, DataLoader

# Assuming 'cleaned_tweet' is the column containing cleaned tweets and 'labels' is the column containing labels
X = train_df['clean_tweet'].tolist()
y = train_df['label'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenization using BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=True)

# Tokenize and pad the sequences
max_len = 50
tokenized_train = tokenizer(X_train, padding=True, truncation=True, max_length=max_len, return_tensors="pt")
tokenized_test = tokenizer(X_test, padding=True, truncation=True, max_length=max_len, return_tensors="pt")

# Convert labels to PyTorch tensors
labels_train = torch.tensor(y_train)
labels_test = torch.tensor(y_test)

# Create TensorDatasets
train_dataset = TensorDataset(tokenized_train['input_ids'], tokenized_train['attention_mask'], labels_train)
test_dataset = TensorDataset(tokenized_test['input_ids'], tokenized_test['attention_mask'], labels_test)

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=3)

from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Training loop
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for i, batch in enumerate(train_dataloader, 1):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        # Print values for each iteration
        if i % 10 == 0:  # Print every 10 iterations
            print(f"Epoch {epoch + 1}/{num_epochs}, Iteration {i}/{len(train_dataloader)}, Loss: {loss.item()}")

    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Average Training Loss: {average_loss}")

# Save the trained BERT model
model.save_pretrained('/content/drive/MyDrive/AD/bert_model_5_it1')
# Evaluation
model.eval()
total_correct = 0
total_samples = 0

with torch.no_grad():
    for i, batch in enumerate(test_dataloader, 1):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)

        total_correct += torch.sum(predictions == labels).item()
        total_samples += labels.size(0)

        # Print values for each iteration
        if i % 10 == 0:  # Print every 10 iterations
            print(f"Iteration {i}/{len(test_dataloader)}, Total Correct: {total_correct}, Total Samples: {total_samples}")

accuracy = total_correct / total_samples
print(f"Test Accuracy: {accuracy}")

Epoch 1/5, Iteration 10/140, Loss: 0.7073901295661926
Epoch 1/5, Iteration 20/140, Loss: 0.28569695353507996
Epoch 1/5, Iteration 30/140, Loss: 0.585030734539032
Epoch 1/5, Iteration 40/140, Loss: 0.182840034365654
Epoch 1/5, Iteration 50/140, Loss: 0.20838436484336853
Epoch 1/5, Iteration 60/140, Loss: 0.39917057752609253
Epoch 1/5, Iteration 70/140, Loss: 0.21597179770469666
Epoch 1/5, Iteration 80/140, Loss: 0.521266520023346
Epoch 1/5, Iteration 90/140, Loss: 0.3243848383426666
Epoch 1/5, Iteration 100/140, Loss: 0.31612908840179443
Epoch 1/5, Iteration 110/140, Loss: 0.5842287540435791
Epoch 1/5, Iteration 120/140, Loss: 0.5296448469161987
Epoch 1/5, Iteration 130/140, Loss: 0.4187641739845276
Epoch 1/5, Iteration 140/140, Loss: 0.38757041096687317
Epoch 1/5, Average Training Loss: 0.4021669771522284
Epoch 2/5, Iteration 10/140, Loss: 0.7592751383781433
Epoch 2/5, Iteration 20/140, Loss: 0.28351572155952454
Epoch 2/5, Iteration 30/140, Loss: 0.46551793813705444
Epoch 2/5, Iteratio

Prediction section for GPT bert model

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load the saved BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=3)
model.load_state_dict(torch.load('/content/drive/MyDrive/AD/bert_model_5_it.pt'))
model.eval()


def remove_stop_words(text):
    # Load the stop words from the file into a set
    with open('/content/drive/MyDrive/Colab Notebooks/stop_words.txt', 'r', encoding='utf-8') as file:
        stop_words = set(file.read().split())

    # Remove stop words
    return ' '.join([word for word in text.split() if word not in stop_words])

# Example input phrase
input_phrase = "آيت الطالب قدم عرض أمام مجلس الحكومة حول تنزيل ورش إصلاح المنظومة الصحية	"


def preprocess_input(input_text):
    # Apply preprocessing steps
    text_without_emoji = replace_emoji(input_text)
    lowercase_text = text_without_emoji.lower()
    text_without_special_chars = remove_special_characters(lowercase_text)
    text_without_stop_words = remove_stop_words(text_without_special_chars)
    # Add any other preprocessing steps here

    return text_without_stop_words


# Preprocess the input phrase
preprocessed_input = preprocess_input(input_phrase)

print("Input Phrase after Preprocessing:", preprocessed_input)

# Tokenize and preprocess the input phrase
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=True)
tokenized_input = tokenizer(preprocessed_input, padding=True, truncation=True, max_length=50, return_tensors="pt")

# Make predictions
with torch.no_grad():
    logits = model(**tokenized_input).logits
    predictions = torch.argmax(logits, dim=1)

# Interpret the predictions
class_labels = ['Class 0', 'Class 1', 'Class 2']
predicted_class = class_labels[predictions.item()]

print(f"The predicted class for the input phrase is: {predicted_class}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Input Phrase after Preprocessing: آيت الطالب قدم عرض أمام مجلس الحكومة تنزيل ورش إصلاح المنظومة الصحية
The predicted class for the input phrase is: Class 1


# **LSTM**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

# Assuming train_df is your original DataFrame
# Split the data into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(train_df['clean_tweet'].values,
                                                                    pd.get_dummies(train_df['label']).values,
                                                                    test_size=0.2, random_state=42)


In [ ]:

# Tokenize and pad sequences for training data
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)

tokenizer.fit_on_texts(train_data)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X_train = tokenizer.texts_to_sequences(train_data)
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of training data tensor:', X_train.shape)

Y_train = train_labels
print('Shape of training label tensor:', Y_train.shape)

# Tokenize and pad sequences for testing data
X_test = tokenizer.texts_to_sequences(test_data)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of testing data tensor:', X_test.shape)

Y_test = test_labels
print('Shape of testing label tensor:', Y_test.shape)

In [ ]:
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.layers import Dropout, Bidirectional
import torch

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

epochs = 6
batch_size = 32
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 250, 100)          5000000   
                                                                 
 spatial_dropout1d_1 (Spati  (None, 250, 100)          0         
 alDropout1D)                                                    
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense_1 (Dense)             (None, 3)                 303       
                                                                 
Total params: 5080703 (19.38 MB)
Trainable params: 5080703 (19.38 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/6
126/126 [==============================] - 133s 1s/step - loss: 0

In [ ]:
# Save the Keras model
model.save('/content/drive/MyDrive/AD/lstm2.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import numpy as np

# Load the saved Keras model
loaded_model = load_model('/content/drive/MyDrive/AD/lstm2.h5')

# Assuming you have a new sentence
new_sentence = "بيدرو سانشيز فمجلس النواب الإسباني قرار دعم مبادرة الحكم الذاتي جاي إرادة إسبانيا ورغبتها فحل ملف الصحرا"

# Tokenize and pad the new sentence using the existing tokenizer
sequences = tokenizer.texts_to_sequences([new_sentence])
padded_sequence = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

# Make a prediction
predictions = loaded_model.predict(np.array(padded_sequence))

# Get the predicted class
predicted_class = np.argmax(predictions)

print("Predicted Class:", predicted_class)